In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:

links_proyectos = []
titulos_proyectos = []
anos_proyectos = []

n = 1

while n <= 12:

    link = 'http://www.fap.df.gov.br/category/editais/' + '?page=' + str(n)

    parserT = parser(link)
    # link
    links = parserT.xpath('//div[@class="row"]/div[@class="col-md-9 col-md-offset-1"]//h4/a/@href')
    #titulos
    titulos = parserT.xpath('//div[@class="row"]/div[@class="col-md-9 col-md-offset-1"]//h4//text()')
    #anos
    for tit in titulos:
        ano = tit.split()[-1]
        if ano.isdigit():
            anos_proyectos.append(tit.split()[-1])
        else:
            anos_proyectos.append('')
        
    links_proyectos+=links
    titulos_proyectos+=titulos
    
    
    n+=1

In [4]:

pdfs_proyectos = []
descripcion_proyectos = []

for link in links_proyectos:
    parserT = parser(link)
    
    # pdfs

    pdfs = parserT.xpath('//p//a/@href')
    pdfs_proyecto = ''

    for pdf in pdfs:
        if pdf.endswith('.pdf'):
            pdfs_proyecto = pdfs_proyecto + pdf + ', '
    
    # Descripción
    descripcion_bruta = parserT.xpath('//p/text()')
    descripcion = ''
    for i in descripcion_bruta:
        i = i.strip()
        if len(i) != 0:
            descripcion = descripcion + i + ' '

    pdfs_proyectos.append(pdfs_proyecto.strip(', '))
    descripcion_proyectos.append(descripcion.strip(' '))

In [6]:
fapdf = pd.DataFrame()

fapdf['Titulo'] = titulos_proyectos
fapdf['Descripción'] = descripcion_proyectos
fapdf['Ano'] = anos_proyectos
fapdf['Link'] = links_proyectos
fapdf['Pdfs'] = pdfs_proyectos
fapdf.to_excel('brasil_fapdf.xlsx')